# Spark Streaming para Elasticsearch

In [ ]:
import os
import urllib.request
import zipfile
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

In [ ]:
#!pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

#### Conexão do Elasticsearch por padrão, nos conectamos ao elasticsearch: 9200, como estamos executando este notebook no Spark-Node, precisamos usar 'elasticsearch-node' em vez de 'localhost', pois esse é o nome do docker container executando o Elasticsearch. Se o índice de teste de fluxo existir, limpe-o e crie um novo.

In [ ]:
es = Elasticsearch('elasticsearch-node:9200')

if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    
    body={
        'mappings': {
            'properties': {
                'count': {'type': 'text'},
                'name': {'type': 'text'},
                'value': {'type': 'text'},
                'timestamp': {'type': 'text'}
            }
        }
    }
    
    es.indices.create(index='stream-test', body=body)
    
print(es.info())

#### Precisamos garantir que o conector ES-Hadoop esteja no caminho de classe do driver; 
#### SparkContext -- É o portão de entrada da funcionalidade Apache Spark.

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'

sc = SparkContext(appName="PythonSparkStreaming")
sc.setLogLevel("WARN")

#### StreamingContext -- É o ponto de entrada para todas as funcionalidades do Spark Streaming (representa a conexão com um cluster Spark e pode ser usado para criar várias fontes de entrada).

In [ ]:
#ssc = StreamingContext(sc, 3)

#### Transmitiremos todos os arquivos gravados no diretório de amostra. Isso está sendo bombeado com dados aleatórios. Gravar o RDD gerado por cada operação em lote de streaming no Elasticsearch.

#### textFileStream -- Procura dentro da pasta indicada e lê esses dados no DStreams sem considerar nenhum tipo de formato.
#### RDD -- Conjuntos de dados distribuídos resilientes (RDD) é uma estrutura de dados fundamental do Spark. É uma coleção imutável de objetos distribuídos. Cada conjunto de dados no RDD é dividido em partições lógicas, que podem ser computadas em diferentes nós do cluster.
#### PARALLELIZE -- É uma estrutura de dados fundamental do Spark, é uma coleção imutável de objetos distribuídos. Cada conjunto de dados no RDD é dividido em partições lógicas, que podem ser computadas em diferentes nós do cluster.
#### LAMBDA -- Uma função lambda pode receber qualquer número de argumentos, mas pode ter apenas uma expressão. Também conhecidas como funções anônimas, são pequenas funções restritas que não precisam de um nome (ou seja, um identificador).
#### MAP -- A operação Map() se aplica a cada elemento do RDD e retorna o resultado como novo RDD. No mapa, o desenvolvedor pode definir sua própria lógica de negócios customizada. A mesma lógica será aplicada a todos os elementos do RDD.
#### saveAsNewAPIHadoopFile -- Método para salvar o RDD no ElasticSearch.  

In [ ]:
print(os.getcwd())

#----------------------------------------------------------------------------

#stream = ssc.textFileStream('./sample/')
#
#def format_sample(x):
#    data = json.loads(x)
#    data['timestamp'] = datetime.fromtimestamp(data['timestamp']).strftime('%Y/%m/%d %H:%M:%S')
#    data['doc_id'] = data.pop('count')
#    return (data['doc_id'], json.dumps(data))
#
#parsed = stream.map(lambda x: format_sample(x))
#
#def handler(rdd):
#        es_write_conf = {       
#        "es.nodes" : '15369fadfcbe',
#        "es.port" : '9200',
#        "es.resource" : 'stream-test',
#        "es.input.json" : "true",
#        "mapred.reduce.tasks.speculative.execution": "false",
#        "mapred.map.tasks.speculative.execution": "false",
#        "es.mapping.id": "doc_id",
#        }
#
#        rdd.saveAsNewAPIHadoopFile(
#                path='-',
#                outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
#                keyClass="org.apache.hadoop.io.NullWritable",
#                valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
#                conf=es_write_conf)

#----------------------------------------------------------------------------

json_folder_path = ('./sample/')

json_files = [ x for x in os.listdir(json_folder_path ) if x.endswith("json") ]

for json_file in json_files:

    json_file_path = os.path.join(json_folder_path, json_file)

    with open (json_file_path) as f:

        data_dict = json.loads(f.read())  

        data_dict['count'] = data_dict.pop('count')
        
        print(data_dict)
        
        es_write_conf = {
            "es.nodes" : '15369fadfcbe',
            #"es.nodes" : '927d8d22d8c7',
            "es.port" : '9200',
            "es.resource" : 'stream-test',
            "es.input.json" : "yes",
            "es.mapping.id": "count"
        }

        rdd = sc.parallelize(data_dict)
        
        def format_data(x):
            return (data_dict['count'], json.dumps(data_dict))      

        rdd = rdd.map(lambda x: format_data(x))
                
        rdd.saveAsNewAPIHadoopFile(
            path='-',
            outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
            keyClass="org.apache.hadoop.io.NullWritable",
            valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
            conf=es_write_conf
        )
      

#### foreachRDD --É um "operador de saída" no Spark Streaming. Ele permite que você acesse os RDDs subjacentes do DStream para executar ações que fazem algo prático com os dados. Por exemplo, usando o foreachRDD, você pode gravar dados em um banco de dados.

In [ ]:
#parsed.foreachRDD(lambda rdd: handler(rdd))

#### Solicitar que também seja impresso em 'stdout'.

In [ ]:
#parsed.pprint()

#### Começar o contexto do spark.

In [ ]:
#ssc.start()

#### Parar o contexto.

In [ ]:
#ssc.stop()

# Bulk Processing ES with Spark

In [ ]:
sc.stop()
sc = SparkContext(appName="PythonSparkReading")  
sc.setLogLevel("WARN")

In [ ]:
es_read_conf = {
    "es.nodes" : '15369fadfcbe',
    #"es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test',
    "es.mapping.id": "doc_id",
    "es.mapping.id": "count"
    }

#### Obtenha um RDD para um determinado arquivo Hadoop com uma nova API InputFormat arbitrária e opções de configuração extras para passar para o formato de entrada.
#### newAPIHadoopRDD -- 

In [ ]:
es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_read_conf)

#### Retornar os 5 primeiros documentos, numa lista com id(count) e o resto da informação.

In [ ]:
es_rdd.take(5)

#### Vamos converter essas tuplas em JSON puro.

In [ ]:
es_rdd = es_rdd.map(lambda x: x[1])

#### Retornar um unico elemento, numa lista com todas as suas informações.

In [ ]:
es_rdd.take(1)

#### Converter o RDD em um Dataframe Spark SQL para que possamos tratá-lo mais como um objeto Pandas

In [ ]:
from pyspark.sql import SparkSession, SQLContext, Row

spark = SparkSession \
    .builder \
    .appName("Spark SQL") \
    .getOrCreate()

In [ ]:
df = es_rdd.map(lambda l: Row(**dict(l))).toDF()

In [ ]:
df.take(1)

In [ ]:
df \
    .groupby('value') \
    .count() \
    .collect()

In [ ]:
df \
    .filter(df.name == 'Legolas')\
    .take(5)

In [ ]:
sc.stop()